In [1]:
import pandas as pd

file_in = "CC_Classified_DAOPHOT_July242023.csv"
csv = pd.read_csv(file_in)

csv.columns

Index(['Index', 'RA', 'DEC', 'x', 'y', 'f090w', 'f200w', 'f335m', 'f444w',
       'f470n', 'e_f090w', 'e_f200w', 'e_f335m', 'e_f444w', 'e_f470n',
       'f090w-f444w', 'f090w-f200w', 'δ_f090w-f200w', 'slope_f090w-f200w',
       'f200w-f335m', 'δ_f200w-f335m', 'slope_f200w-f335m', 'f335m-f444w',
       'δ_f335m-f444w', 'slope_f335m-f444w', 'f444w-f470n', 'δ_f444w-f470n',
       'slope_f444w-f470n', '(f090w-f200w)-(f200w-f444w)',
       'δ_(f090w-f200w)-(f200w-f444w)', 'Sum1', 'e_f090w-f444w',
       'e_f090w-f200w', 'e_δ_f090w-f200w', 'e_slope_f090w-f200w',
       'e_f200w-f335m', 'e_δ_f200w-f335m', 'e_slope_f200w-f335m',
       'e_f335m-f444w', 'e_δ_f335m-f444w', 'e_slope_f335m-f444w',
       'e_f444w-f470n', 'e_δ_f444w-f470n', 'e_slope_f444w-f470n',
       'e_(f090w-f200w)-(f200w-f444w)', 'e_δ_(f090w-f200w)-(f200w-f444w)',
       'e_Sum1', 'Class_PRF', 'Prob_PRF', 'Class_RF', 'Prob_RF', 'Init_Class',
       'Survey', 'SPICY_ID'],
      dtype='object')

In [2]:
columns = ['RA', 'DEC', 'f090w', 'f200w', 'f335m', 'f444w','f470n', 'Prob_PRF','Prob_RF']#'f187n',

new_cols = ['RA', 'DEC', 'F090w','F200w','F335m','F444w','F470n',  'Prob YSO PRF', 'Prob YSO RF'] #'F187n','F770w', 'F1130w', 'F1280w', 'F1800w',


In [3]:

file_out = file_in.split('.')[0]+'_latex.txt'
print("Printing to ", file_out)
f = open(file_out,'w')

for j in range(0,len(columns)):
    f.write(new_cols[j]+'&')
f.write('\\\ \n')
for i in range(0,len(csv)):
    for j in range(0,len(columns)):
        str_tmp = str("%.4f" %csv[columns[j]][i])+"&"
        f.write(str_tmp)
    f.write('\\\ \n')

f.close()

Printing to  CC_Classified_DAOPHOT_July242023_latex.txt


In [4]:
csv_yso = csv[(csv.Class_RF==0)|(csv.Class_PRF==0)].reset_index()
csv_yso = csv_yso.loc[csv_yso.isnull().sum(1).sort_values(ascending=1).index,columns]


file_out = file_in.split('.')[0]+'_latex_ysos.txt'
print("Printing to ", file_out)


f = open(file_out,'w')

for j in range(0,len(columns)):
    f.write(new_cols[j]+'&')
f.write('\\hline \\\ \n')
for i in range(0,len(csv_yso)):
    for j in range(0,len(columns)):
        str_tmp = str("%.4f" %csv_yso[columns[j]].values[i])+"&"
        f.write(str_tmp)
    f.write('\\\ \n')

f.close()

Printing to  CC_Classified_DAOPHOT_July242023_latex_ysos.txt


In [16]:
from astropy.io import fits
import astropy.wcs as wcs
from astropy.coordinates import match_coordinates_sky, SkyCoord
from astropy.table import Table
import astropy.units as u
from astropy.io.votable import from_table, writeto

xray1 = Table(fits.open('Archive/J_A+A_564_A120_table1.dat.fits')[1].data).to_pandas()
xray2 = Table(fits.open('Archive/J_A+A_564_A120_table3.dat.fits')[1].data).to_pandas()
xray2[xray2.Src>0].Seq
xray1 = xray1.loc[xray2[xray2.Src>0].index].reset_index()
# tmp_votab = from_table(Table.from_pandas(xray1,units={'RA':u.deg,'DEC':u.deg}))

/Users/breannacrompvoets/miniforge3/envs/SF/lib/python3.10/site-packages/astropy/io/fits/column.py:831: VerifyWarning: It is strongly recommended that column names contain only upper and lower-case ASCII letters, digits, or underscores for maximum compatibility with other software (got '[3.4]').
  warnings.warn(
/Users/breannacrompvoets/miniforge3/envs/SF/lib/python3.10/site-packages/astropy/io/fits/column.py:831: VerifyWarning: It is strongly recommended that column names contain only upper and lower-case ASCII letters, digits, or underscores for maximum compatibility with other software (got '[4.5]').
  warnings.warn(
/Users/breannacrompvoets/miniforge3/envs/SF/lib/python3.10/site-packages/astropy/io/fits/column.py:831: VerifyWarning: It is strongly recommended that column names contain only upper and lower-case ASCII letters, digits, or underscores for maximum compatibility with other software (got '[5.8]').
  warnings.warn(
/Users/breannacrompvoets/miniforge3/envs/SF/lib/python3.10

In [32]:
print(xray1.columns)
tol=0.001
csv_y = csv[csv.Prob_PRF>0.5].reset_index()
ind, sep, _ = match_coordinates_sky(SkyCoord(xray1.RAdeg,xray1.DEdeg,unit=u.deg),SkyCoord(csv_y.RA,csv_y.DEC,unit=u.deg))


Index(['index', 'Seq', 'Name', 'RAdeg', 'DEdeg', 'ePos', 'theta', 'Ctnet',
       'e_Ctnet', 'Bt', 'CHnet', 'fPSF', 'Signi', 'l_logPB', 'logPB', 'f_Seq',
       'Var', 'Emed'],
      dtype='object')


In [33]:
dao_xray = csv_y.loc[ind[sep<tol*u.deg],['RA','DEC','Prob_PRF','Prob_RF']]
print(len(ind),print(len(dao_xray),print(len(xray1))))
print(xray1.Seq)
dao_xray['Name'] = ['X-RAY '+ str(xray1.loc[i,'Seq']) for i in xray1[sep<tol*u.deg].index]


500
14 None
500 None
0        1
1        2
2        3
3        5
4        7
      ... 
495    675
496    676
497    677
498    678
499    679
Name: Seq, Length: 500, dtype: int16


In [34]:
dao_xray.sort_values('RA')

,RA,DEC,Prob_PRF,Prob_RF,Name
124,159.203575,-58.634598,0.999840,0.978,X-RAY 27
322,159.218300,-58.607508,0.998634,0.970,X-RAY 34
15,159.222348,-58.674431,0.506863,0.468,X-RAY 35
312,159.223134,-58.609556,0.906611,0.714,X-RAY 36
3,159.224662,-58.681911,0.554947,0.446,X-RAY 39
197,159.228207,-58.625727,0.862667,0.596,X-RAY 40
411,159.243539,-58.583399,0.844379,0.806,X-RAY 51
396,159.244148,-58.587722,0.788723,0.578,X-RAY 53
395,159.244762,-58.588353,0.598248,0.544,X-RAY 56
115,159.251428,-58.641576,0.994609,0.998,X-RAY 64


In [35]:

tmp_votab = from_table(Table.from_pandas(dao_xray,units={'RA':u.deg,'DEC':u.deg}))
writeto(tmp_votab,'xray_matched_July242023.xml')

In [36]:
dao_xray.to_csv('xray_matched.csv')

In [40]:
dao_matched = pd.read_csv('matches_to_July242023.csv')
cols = ['YSO_n','Name','Prob','Nearby']

file_out = 'matches_to_July242023_latex_ysos.txt'
print("Printing to ", file_out)


f = open(file_out,'w')

for j in range(0,len(cols)):
    f.write(cols[j]+'&')
f.write(' \\hline \\\ \n')
for i in range(0,len(dao_matched)):
    for j in range(0,len(cols)):
        str_tmp = str(dao_matched[cols[j]].values[i])+"&"
        f.write(str_tmp)
    f.write(' \\\ \n')

f.close()

Printing to  matches_to_July242023_latex_ysos.txt
